<a href="https://colab.research.google.com/github/SupawatDev/path-loss-tester/blob/master/path_loss_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Path Loss Tester
Path Loss tester is a tool to verify the result of electromagnetic propagation loss from the generated data from WCsim.

In [221]:
from google.colab import files
from math import pi, log10, cos, sin, asin
from scipy.signal import normalize
import numpy as np
import pandas as pd
import io
c = 3e8

### Import the data

In [3]:
print("Upload files.")
upload = files.upload()

Upload direct path result.


Saving direct_path.csv to direct_path (1).csv
Saving reflection.csv to reflection.csv
Saving single-edge.csv to single-edge.csv
Saving multiple-edges.csv to multiple-edges.csv
Upload reflection result.


In [134]:
#Load .csv data to Pandas
direct_data = pd.read_csv(io.BytesIO(upload['direct_path.csv']))
ref_data = pd.read_csv(io.BytesIO(upload['reflection.csv']))
single_edge_data = pd.read_csv(io.BytesIO(upload['single-edge.csv']))
multi_edges_data = pd.read_csv(io.BytesIO(upload['multiple-edges.csv']))
direct_data.head()

,tx_x_pos,tx_y_pos,tx_z_pos,tx_freq,rx_x_pos,rx_y_pos,rx_z_pos,distance,path_loss_dB
0,0,5,0,2.300000e+09,34,1.5,-59,68.1854,76.3502
1,0,5,0,2.300000e+09,24,1.5,0,24.2539,67.3719
2,0,5,0,2.300000e+09,42,1.5,-9,43.0958,72.3650
3,0,5,0,2.300000e+09,-5,1.5,16,17.1245,64.3487
4,0,5,0,2.300000e+09,12,1.5,38,40.0031,71.7182


## Direct Path Verification

\begin{equation}
Loss_{direct} = (\frac{4\pi d_1}{\lambda})^2
\end{equation}

In [194]:
# Function for calculating direct path loss of radio wave propagation.
def DirectPath(tx_positions, frequencies, rx_positions):
  distance = np.sum((tx_positions-rx_positions)**2, axis=1)
  distance = np.sqrt(distance)
  wave_length = c/frequencies
  distance = np.reshape(distance, (-1,1))
  wave_length = np.reshape(wave_length, (-1,1))
  path_loss = (4*pi*distance/wave_length)**2
  return 10*np.log10(path_loss)

In [298]:
tx_positions = direct_data.iloc[:,0:3].to_numpy()
tx_frequencies = direct_data.iloc[:,3:4].to_numpy()
rx_positions = direct_data.iloc[:,4:7].to_numpy()
computed_result = DirectPath(tx_positions, tx_frequencies, rx_positions).reshape(-1,1)
reference_result = direct_data.iloc[:,-1].to_numpy().reshape((-1,1))
# Round to 1 digit because the PI value in simulator is not accurate.
check_table = pd.DataFrame( {'com':computed_result.ravel(), 'ref': reference_result.ravel()})
assert (np.round(computed_result, 1) == np.round(reference_result, 1)).all()
print("Direct Path Results are correct.")
check_table.head()

Direct Path Results are correct.


,com,ref
0,76.350158,76.3502
1,67.371948,67.3719
2,72.365033,72.3650
3,64.348709,64.3487
4,71.718207,71.7182


## Reflection Verification

### Reflection Coefficient

\begin{equation}
  \Gamma_{TE} = 
  \begin{cases}
    1 , & \text{ where \( \sqrt{\vert \frac{n_1}{n_2} \vert sin\theta_1 } >=1 \) } \\
         \frac {\sqrt{n_2} cos(\theta_1) - \sqrt{n_2} cos(\theta_2)}
          {\sqrt{n_1} cos(\theta_1) + \sqrt{n_2} cos(\theta_2)} , &  \text{ else }
  \end{cases}
\end{equation}


\begin{equation}
  \Gamma_{TM} =
\end{equation}

In [191]:
ref_data.head()

,tx_x_pos,tx_y_pos,tx_z_pos,tx_freq,rx_x_pos,rx_y_pos,rx_z_pos,ref_x_pos,ref_y_pos,ref_z_pos,tm_or_te,ref_coeff,d1,d2,angle2norm_deg,ref_loss_dB
0,0,5,0,2.300000e+09,34,1.5,-59,36.1217,2.834910,-30.0909,E,-0.646290,47.0630,29.01760,62.83030,81.0933
1,0,5,0,2.300000e+09,34,1.5,-59,26.1838,0.010512,-45.4366,E,-0.912804,52.6779,15.72510,84.56490,77.1703
2,0,5,0,2.300000e+09,24,1.5,0,64.1040,2.846930,0.0000,E,-0.394866,64.1402,40.12660,1.92368,88.1103
3,0,5,0,2.300000e+09,24,1.5,0,18.4833,0.010678,0.0000,E,-0.778438,19.1448,5.71422,74.89300,69.7615
4,0,5,0,2.300000e+09,42,1.5,-9,41.3905,1.596380,-10.0407,E,-0.568004,42.7267,1.20995,53.49560,77.4458


In [205]:
tx_positions = ref_data.iloc[:,0:3].to_numpy()
tx_frequencies = ref_data.iloc[:,3:4].to_numpy()
rx_positions = ref_data.iloc[:,4:7].to_numpy()
ref_data.shape

(194, 16)

In [249]:
def ReflectionCoefficient(tx_positions, rx_positions, angle2norm_deg, n_1, n_2, polar):
  angle_1 = np.radians(angle2norm_deg)

  c1, c2 = c/np.sqrt(n_1), c/np.sqrt(n_2)
  angle_2 = np.arcsin(c2*np.sin(angle_1)/c1)

  if polar == 'TE':
    answer = (np.sqrt(n_1)*np.cos(angle_1) - np.sqrt(n_2)*np.cos(angle_2))/ \
           (np.sqrt(n_1)*np.cos(angle_1) + np.sqrt(n_2)*np.cos(angle_2))

  if polar == 'TM':
    answer = (np.sqrt(n_2)*np.cos(angle_1) - np.sqrt(n_1)*np.cos(angle_2))/ \
           (np.sqrt(n_2)*np.cos(angle_1) + np.sqrt(n_1)*np.cos(angle_2))

  for i in range(len(tx_positions)):
    if np.sqrt(np.abs(n_1/n_2)*np.sin(angle_1[i]) ) >= 1:
      answer[i] = 1

  return answer

In [250]:
angles = ref_data.iloc[:, -2:-1].to_numpy().reshape((-1, 1))

In [299]:
computed_refl_coe = ReflectionCoefficient(tx_positions, rx_positions, angles, 1.0003, 5.31, 'TE' ).reshape((-1,1))
ref_refl_coe = ref_data.iloc[:,-5].to_numpy().reshape((-1,1))
check_table = pd.DataFrame( {'com':computed_refl_coe.ravel(), 'ref': ref_refl_coe.ravel()})
assert (np.round(computed_refl_coe,2) == np.round(ref_refl_coe,2)).all()
print("Reflection Coefficient Results are correct.")
check_table.head()

Reflection Coefficient Results are correct.


,com,ref
0,-0.646290,-0.646290
1,-0.912805,-0.912804
2,-0.394866,-0.394866
3,-0.778438,-0.778438
4,-0.568004,-0.568004


### Reflection Loss
\begin{equation}
Loss_{reflect} = (\frac{4\pi (d_1+d_2)}{\lambda \Gamma})^2
\end{equation}